In [1]:
import numpy as np
import pandas as pd
import numpy as np
from finta import TA
import yfinance as yf
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('final_data.csv')
data

,Unnamed: 0,message,symbol,Date,Pctchange,Open,High,Low,Close,Volume,Dividends,Stock Splits,label,Body,Neg,Neu,Pos
0,97799,aapl anyone needs test gets test beautiful tes...,AAPL,2020-03-11,-0.030124,62.484177,65.916729,60.545737,60.601887,418474000,0.0,0.0,0,aapl anyone needs test gets test beautiful tes...,-0.293482,0.529573,-0.473534
1,2012025,bet mispriced stocks etfs bac nflx urn zig,NFLX,2019-07-18,-0.025665,323.399994,325.850006,314.230011,315.100006,16302500,0.0,0.0,0,bet mispriced stocks etfs bac nflx urn zig,0.139890,-0.541555,0.774006
2,15240,aapl see pretty strong buy signal enter approa...,AAPL,2020-06-25,-0.029582,89.206454,89.429220,86.418213,86.567543,205256800,0.0,0.0,0,aapl see pretty strong buy signal enter approa...,-0.210327,0.174892,-0.084247
3,295121,aapl spy incredible ta guru followers rich,AAPL,2019-03-21,-0.021962,47.073384,47.639693,45.974508,46.039574,169630800,0.0,0.0,0,aapl spy incredible ta guru followers rich,-0.626426,0.787519,-0.182236
4,1441619,amzn macquarie reiterates buy rating,AMZN,2018-07-27,-0.026328,91.366501,91.474998,88.301003,88.960999,131246000,0.0,0.0,0,amzn macquarie reiterates buy rating,-0.150986,0.350798,-0.384577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428926,1432977,amzn point many days happening manipulation al...,AMZN,2018-08-28,0.022857,97.672501,99.934502,97.446999,99.904999,130636000,0.0,0.0,2,amzn point many days happening manipulation al...,-0.155573,0.210340,-0.108832
428927,103126,stack gains aapl,AAPL,2020-03-05,0.024929,68.846384,70.999666,68.658402,70.562660,226176800,0.0,0.0,2,stack gains aapl,-0.432490,0.446864,0.060105
428928,74953,aapl ignore bottoming tails 60min may happen n...,AAPL,2020-04-03,0.046114,61.253721,64.234620,60.882637,64.078377,201820400,0.0,0.0,2,aapl ignore bottoming tails 60min may happen n...,-0.838552,1.006496,-0.101169
428929,520178,aapl huge pump since dip pre market,AAPL,2018-03-23,0.027965,39.894338,41.088296,39.507428,41.009964,150164800,0.0,0.0,2,aapl huge pump since dip pre market,-0.407604,0.501284,-0.110717


In [6]:
data.columns

Index(['Unnamed: 0', 'message', 'symbol', 'Date', 'Pctchange', 'Open', 'High',
       'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'label', 'Body',
       'Neg', 'Neu', 'Pos'],
      dtype='object')

In [10]:
data = pd.read_csv('final_data.csv')[['message', 'symbol', 'Date', 'label', 'Body', 'Neg', 'Neu', 'Pos']]

data['symbol'] = data['symbol'].replace({'FB':'META'})
codes = data['symbol'].unique()
start_date = data['Date'].min()
end_date = data['Date'].max()
res = []


for ticker_symbol in tqdm(codes):
    ticker = yf.Ticker(ticker_symbol)
    df = ticker.history(start=start_date, end=end_date)
    df['symbol'] = ticker_symbol
    res.append(df)

    
price = pd.concat(res).reset_index()
price['Time'] = price['Date'].dt.strftime('%H:%M:%S')
price['Date'] = price['Date'].dt.strftime('%Y-%m-%d')
price['Pctchange'] = (price['Close'] - price['Open'])/price['Open']
price_new = pd.DataFrame(price.values[:,1:], index = price['Date'].values[:], columns = price.columns[1:]).reset_index()
price_new.columns = price.columns
sentiment = pd.merge(data, price_new, left_on=['Date','symbol'], right_on=['Date','symbol'], how='outer')
sentiment = sentiment.sort_values('Date')

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.33it/s]


In [11]:
codes = sentiment.symbol.unique()
res = []
for code in codes:
    df_temp = sentiment[sentiment['symbol']==code]
    df_temp['Pctchange'] = df_temp['Pctchange'].fillna(method='bfill')
    df_temp = df_temp[['message','symbol','Date','Pctchange','Open','High','Low','Close','Volume','Dividends','Stock Splits', 'Neg', 'Neu',
       'Pos']].dropna()
    res.append(df_temp)

In [12]:
data_final = pd.concat(res)

In [13]:
data_final = data_final[(data_final['Date']>='2017-01-01')&(data_final['Date']<='2019-12-31')]

In [17]:
data_final.to_csv('data_2017_2019_with_price.csv')